# Génération du dataset pour l'analyse de données

Les données issues du scraping de l'API IDF Mobilités ne constituent pas un jeu de données en tant que tel. Le rôle de ce notebook est de passer du des données brutes à un jeu de données exploitable par la suite.

In [1]:
# On utilise cudf.pandas pour accélérer les opérations Pandas sur GPU, optionnel

!pip install \
  --extra-index-url=https://pypi.nvidia.com \
  cudf-cu12==24.12.* \
  dask-cudf-cu12==24.12.* \
  cuml-cu12==24.12.* \
  cugraph-cu12==24.12.*

%load_ext cudf.pandas

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [2]:
!pip install -r requirements.txt

In [3]:
import os
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv

load_dotenv()

False

Pour avoir plus de flexibilité pour mener notre analyse, nous ne travaillerons pas seulement avec un dataset combinant les données sur les perturbations et les lignes affectées, nous garderons aussi de côté les données propres aux perturbations et aux lignes séparées.

In [4]:
from scraping.config import ROOT
import s3fs

# Le code en commentaire sert à accéder au bucket SSP Cloud qui contient les donénes de scraping en n'étant pas Théo Lartigau
# Il nécessite que les variables d'environnement MINIO_KEY et MINIO_SECRET soient enregistrées

# KEY = os.environ.get("MINIO_KEY")
# SECRET = os.environ.get("MINIO_SECRET")

# fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}, key=KEY, secret=SECRET)

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

paths = fs.ls(ROOT)

paths = [p for p in paths if not p.split("/").pop().startswith(".")] # on exclue les fichiers cachés

all_results = []
all_disruptions = []
all_lines = []

for file_path in paths:
    with fs.open(file_path, 'r', encoding='ascii') as f:
        raw_data = f.read()
        data = json.loads(raw_data)
        
        last_updated = data.get('lastUpdatedDate')
        disruptions = data.get('disruptions', [])
        lines = data.get('lines', [])
        
        all_results.append({
            'lastUpdatedDate': last_updated,
            'disruptions': disruptions,
            'lines': lines
        })
        
        for d in disruptions:
            all_disruptions.append({
                'disruption_id': d.get('id'),
                'applicationPeriods': d.get('applicationPeriods'),
                'lastUpdate': d.get('lastUpdate'),
                'cause': d.get('cause'),
                'severity': d.get('severity'),
                'title': d.get('title'),
                'message': d.get('message'),
                'file_lastUpdatedDate': last_updated
            })
        
        for l in lines:
            all_lines.append({
                'line_id': l.get('id'),
                'name': l.get('name'),
                'shortName': l.get('shortName'),
                'mode': l.get('mode'),
                'networkId': l.get('networkId'),
                'impactedObjects': l.get('impactedObjects', []),
                'file_lastUpdatedDate': last_updated
            })

Du fait de perturbations pouvant être de longue durée (travaux...), nos données brutes comportent de nombreux duplicats que nous allons devoir traiter.

In [5]:
df_results = pd.DataFrame(all_results)
df_disruptions = pd.DataFrame(all_disruptions)
df_lines = pd.DataFrame(all_lines)

results_before = len(df_results)
disruptions_before = len(df_disruptions)
lines_before = len(df_lines)

print("Total results (before):", results_before)
print("Total disruptions (before):", disruptions_before)
print("Total lines (before):", lines_before)

df_lines.to_csv("data/lines.csv") # Nous aurons besoin de toutes les données sur les lignes pour faire une jointure avec les pertubations

df_results = df_results.drop_duplicates(subset=['lastUpdatedDate'])
df_disruptions = df_disruptions.drop_duplicates(subset=['disruption_id'])
df_lines_deduplicated = df_lines.drop_duplicates(subset=['line_id'])

results_after = len(df_results)
disruptions_after = len(df_disruptions)
lines_after = len(df_lines_deduplicated)

print("Total results (after):", results_after)
print("Total disruptions (after):", disruptions_after)
print("Total lines (after):", lines_after)

df_results.to_csv("data/results.csv")
df_disruptions.to_csv("data/disruptions.csv")
df_lines_deduplicated.to_csv("data/lines_unique.csv")

column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupport

In [6]:
print("Total results (%reduction):", (1 - round(results_after/results_before, 4)) * 100)
print("Total disruptions (%reduction):", (1 - round(disruptions_after/disruptions_before, 4)) * 100)
print("Total lines (%reduction):", (1 - round(lines_after/lines_before, 4)) * 100)

Total results (%reduction): 0.0
Total disruptions (%reduction): 98.85000000000001
Total lines (%reduction): 99.92999999999999


On comprend ici que l'API IDF Mobilités est mise à jour plus régulièrement que notre fréquence de scraping (aucun doublon dans les résultats des appels API). Cela veut dire qu'il n'est pas impossible que nous ayons manqué des perturbations de très courte durée sur la période considérée. Nous garderons cela en tête pour l'analyse des données.

La déduplication a toutefois été très importante pour les données sur les perturbations et sur les lignes, ce qui était attendu. Avec 19757 perturbations différentes dans notre jeu de données établi sur 3 semaines. Nous avons assez de données pour faire une analyse intéressante, bien que l'idéal serait de produire un outil permettant une analyse continue et automatisée des perturbations fournies par l'API.

In [9]:
# jointure entre lignes et perturbations
def merge_lines_disruptions(df_lines, df_disruptions):
    line_disruption_links = []

    for idx, row in df_lines.iterrows():
        impacted_objects = row['impactedObjects']
        line_id = row['line_id']
        if impacted_objects:
            for obj in impacted_objects:
                disruptions_ids = obj.get('disruptionIds', [])
                for d_id in disruptions_ids:
                    line_disruption_links.append({
                        'line_id': line_id,
                        'disruption_id': d_id
                    })

    df_line_disruption = pd.DataFrame(line_disruption_links)

    df_line_disruption = df_line_disruption.merge(df_lines, on='line_id') \
                                        .merge(df_disruptions, left_on='disruption_id', right_on='disruption_id')

    return df_line_disruption

In [10]:
df_line_disruption = merge_lines_disruptions(df_lines, df_disruptions)

df_line_disruption.to_csv("data/line_disruption.csv")

column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24


: 

In [25]:
len(df_line_disruption)

column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24
column_view::get_data: Unsupported type: 24


5245